In [1]:
import requests
import json
from datetime import date

In [2]:
import re

def sanitize_isbn_list(data: dict) -> list:
    """Takes a JSON-like dict with a list of ISBNs and returns a sanitized list of 13-digit strings."""
    raw_isbns = data.get("isbn", [])
    clean_isbns = []

    for isbn in raw_isbns:
        digits_only = re.sub(r"[^\d]", "", isbn)  # remove dashes and non-digit chars
        if len(digits_only) == 13:
            clean_isbns.append(digits_only)
        else:
            print(f"Warning: Skipping invalid ISBN '{isbn}' → '{digits_only}'")

    return clean_isbns


In [3]:
input_data = {
    "isbn": [
        "978-1-5098-6739-1",
        "978-0-679-76288-1",
        "978-2-07-036822-8",
        "978-0-141-03613-7"
    ]
}

clean_isbns = sanitize_isbn_list(input_data)
print(clean_isbns)


['9781509867391', '9780679762881', '9782070368228', '9780141036137']


In [4]:
def fetch_book_metadata(isbn: str) -> dict:
    url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"
    response = requests.get(url)
    data = response.json()

    if "items" not in data:
        raise ValueError(f"No book found for ISBN: {isbn}")

    book_info = data["items"][0]["volumeInfo"]

    return {
        "title": book_info.get("title", "Unknown Title"),
        "author": ", ".join(book_info.get("authors", ["Unknown Author"])),
        "isbn": isbn,
        "status": "to read",
        "category": "professional reading",
        "pages": book_info.get("pageCount", None),
        "publication_date": book_info.get("publishedDate", ""),
        "cover_url": book_info.get("imageLinks", {}).get("thumbnail", ""),
        "added_on": str(date.today())
    }


In [5]:

if __name__ == "__main__":
    isbn_input = '9781509867391'
    metadata = fetch_book_metadata(isbn_input)
    print(json.dumps(metadata, indent=2))

{
  "title": "Talk Like TED",
  "author": "Carmine Gallo",
  "isbn": "9781509867391",
  "status": "to read",
  "category": "professional reading",
  "pages": 288,
  "publication_date": "2017-07-13",
  "cover_url": "http://books.google.com/books/content?id=Q97xAQAACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api",
  "added_on": "2025-06-16"
}
